In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

%matplotlib inline
pd.set_option('display.max_columns', 100)

# Model

In [ ]:
class MLP() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
    def day_MLP(self, X_input) :
        layer = tf.layers.dense(X_input, self.emb_dim)
        
        for idx in range(self.num_layer//3+1) :
            norm1 = tf.contrib.layers.layer_norm(layer)
            relu1 = tf.nn.leaky_relu(norm1)
            layer1 = tf.layers.dense(relu1, self.emb_dim*2)
                
            norm2 = tf.contrib.layers.layer_norm(layer1)
            relu2 = tf.nn.leaky_relu(norm2)
            layer2 = tf.layers.dense(relu2, self.emb_dim)
            
            layer = layer2 + layer
                
        summary_layer = tf.layers.dense(layer, self.emb_dim//2)
        return summary_layer
        
    
    def build(self, batch_size, input_dim, emb_dim, output_dim, num_layer, num_unit, activation) :
        with tf.variable_scope(self.name) :
            ## Setting ##
            # input  : ? x input_length x input_dim
            self.X = tf.placeholder(tf.float32, [None, input_dim])
            self.Y = tf.placeholder(tf.float32, [None, output_dim])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            self.batch_size = batch_size
            self.day_dim = input_dim//8
            self.emb_dim = emb_dim
            self.output_dim = output_dim
            self.num_layer = num_layer
            self.num_unit = num_unit
            self.activation = activation
            #############
            
            
            ## MLP ##
            day1 = self.day_MLP(self.X[:,:self.day_dim])
            day2 = self.day_MLP(self.X[:,self.day_dim : self.day_dim*2])
            day3 = self.day_MLP(self.X[:,self.day_dim*2 : self.day_dim*3])
            day4 = self.day_MLP(self.X[:,self.day_dim*3 : self.day_dim*4])
            day5 = self.day_MLP(self.X[:,self.day_dim*4 : self.day_dim*5])
            day6 = self.day_MLP(self.X[:,self.day_dim*5 : self.day_dim*6])
            day7 = self.day_MLP(self.X[:,self.day_dim*6 : self.day_dim*7])
            day8 = self.day_MLP(self.X[:,self.day_dim*7 :])
            
            day_total = tf.concat([day1,day2,day3,day4,day5,day6,day7,day8], axis=1)
            day_norm = tf.contrib.layers.layer_norm(day_total)
            day_relu = tf.nn.leaky_relu(day_norm)
            layer = tf.layers.dense(day_relu, self.num_unit)
            
            layer_lst = []
            layer_lst.append(layer)
            
            for idx in range((self.num_layer-2)//2) :
                dense_layer = tf.concat(layer_lst, axis=1)
                
                norm1 = tf.contrib.layers.layer_norm(dense_layer)
                relu1 = tf.nn.leaky_relu(norm1)
                layer1 = tf.layers.dense(relu1, self.num_unit*2)
                
                norm2 = tf.contrib.layers.layer_norm(layer1)
                relu2 = tf.nn.leaky_relu(norm2)
                layer2 = tf.layers.dense(relu2, self.num_unit)
                dropout = tf.layers.dropout(layer2, training=self.training)
                
                layer_lst.append(dropout)
                
            layer_lst2 = []
            concat_layer = tf.concat(layer_lst, axis=1)
            batch, hidden = concat_layer.get_shape().as_list()
            
            compression_norm = tf.contrib.layers.layer_norm(concat_layer)
            compression_relu = tf.nn.leaky_relu(compression_norm)
            compression_layer = tf.layers.dense(compression_relu, hidden//2)
            layer_lst2.append(compression_layer)
            
            for idx in range((self.num_layer-2)//2) :
                dense_layer2 = tf.concat(layer_lst2, axis=1)
                
                norm1 = tf.contrib.layers.layer_norm(dense_layer2)
                relu1 = tf.nn.leaky_relu(norm1)
                layer1 = tf.layers.dense(relu1, self.num_unit*2)
                
                norm2 = tf.contrib.layers.layer_norm(layer1)
                relu2 = tf.nn.leaky_relu(norm2)
                layer2 = tf.layers.dense(relu2, self.num_unit)
                dropout = tf.layers.dropout(layer2, training=self.training)
                
                layer_lst2.append(dropout)
            
            concat_layer2 = tf.concat(layer_lst2, axis=1)
            compression_norm2 =tf.contrib.layers.layer_norm(concat_layer2)
            compression_relu2 = tf.nn.leaky_relu(compression_norm2)
            compression_layer2 = tf.layers.dense(compression_relu2, self.num_unit)
            layer = tf.layers.dense(compression_layer2, self.output_dim)
            #########################
            
            
            ## Classifier ##
            self.logit = layer
            self.softmax = tf.nn.softmax(self.logit)
            self.softmax_logit = tf.nn.softmax_cross_entropy_with_logits(logits=self.logit, labels=self.Y)
            ################
            
            
            ## Learning ##
            self.cost =  tf.reduce_mean(self.softmax_logit)

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            
            self.prediction = tf.equal(tf.argmax(self.logit, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(self.prediction, tf.float32))    
            ##############
        
        
    def train(self, X_input, Y_input, learning_rate, training=True):
        feed_dict = {self.X: X_input, self.Y: Y_input, self.learning_rate: learning_rate, self.training: training}
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict=feed_dict)
        
        return _, cost
    
    def predict(self, X_input, training=False):
        feed_dict = {self.X: X_input, self.training: training}
        result = self.sess.run([self.logit], feed_dict=feed_dict)
            
        return result
    
    def evaluate(self, X_input, Y_input):
        size = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for idx in range(0, size, self.batch_size):
            X_batch = X_input[idx:idx + batch_size]
            Y_batch = Y_input[idx:idx + batch_size]
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= size
        total_acc /= size
            
        return total_loss, total_acc